In [2]:
%matplotlib inline
import tensorflow as tf
from vahun.Text import Text
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
from vahun.autoencoder import Autoencoder_ffnn
from vahun.tools import show_performance

timer=Timer()

corpus_path='/mnt/store/velkey/mnsz2/mnsz2.100M.freqs.hunfiltered_lowered'
encode=180
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

corpus=Text(corpus_path=corpus_path,size=100000)

logger=explog(encoder_type="hunfiltered_lowered_uniq_"+str(encode),
              encoding_dim=encode,
              feature_len=20,
              lang=corpus_path,
              unique_words=len(set(corpus.wordlist)),
              name="autoencoder_lowered_"+str(encode),
              population_size=0,
              words=len(corpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [ ]:
encoder=Autoencoder_ffnn(experiment=exp,
                 logger=logger,tf_session=sess,
                 inputdim=len(corpus.abc)*10,
                 layerlist=exp.weights,
                 encode_index=int(exp.len/2),corpus=corpus,
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid,
                 charnum=len(corpus.abc))

In [ ]:
encoder.train(corpus.x_train,corpus.x_valid,corpus.x_test,512,30)

In [ ]:
stds=show_performance(encoder,corpus.x_test,corpus,10,printer=True,inputfsize=len(corpus.abc))

In [ ]:
std=show_performance(encoder,["eh","kecske","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],corpus,printer=True,inputfsize=len(corpus.abc))

In [ ]:
def decode_critical(lista,enc=180):
    for POS in lista:
        encoded=np.ones(enc)*-10
        encoded[POS]=10
        a=encoder.decode([encoded])[0].reshape([10,len(corpus.abc)])
        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)

In [ ]:
import heapq
topstd=heapq.nlargest(6, range(len(stds)), stds.__getitem__)
big_stuff=[]
for i in range(50):
    big_stuff.append([])
    for topind in topstd:
        a=encoder.encode([corpus.x_test[i]])
        a[0][topind]=2
        b=encoder.decode([a[0]])[0].reshape([10,len(corpus.abc)])
        c=corpus.defeaturize_data_charlevel_onehot([b])
        big_stuff[i].append(c)
for row in big_stuff:
    print(row)